<a href="https://colab.research.google.com/github/mustafaikizler/Data_Analytics_Project/blob/Fernando/Final_Project_DataBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import psycopg2

In [ ]:
conn = psycopg2.connect(
    host="dataviz.ce3i6u4ewfvq.us-east-2.rds.amazonaws.com",
    database="UofT_DB",
    user="postgres",
    password="postgres")
cursor = conn.cursor()

In [ ]:
# Read in data from S3 Buckets - Owners
Owners_df = pd.read_csv('https://uoftfinalproject.s3.us-east-2.amazonaws.com/Clients.csv')


#Cleand Data
Owners_df= Owners_df.dropna()
Owners_df

,ID,Ident,sex,bdate,status,marital,region,city,country
3,AA12541,AA,female,1934-05-04,active,Married,ON,Millbank,Canada
4,AA12707,AA,female,1934-10-17,active,,ON,Stratford,Canada
5,AA12759,AA,female,1934-12-08,active,Widowed,ON,London,Canada
7,AA12913,AA,female,1935-05-11,active,Widowed,ON,Leamington,Canada
9,AA13086,AA,female,1935-10-31,active,Widowed,ON,London,Canada
...,...,...,...,...,...,...,...,...,...
123435,ZW25694,ZW,male,1970-05-08,active,Married,ON,Kingsville,Canada
123437,ZW28059,ZW,male,1976-10-28,active,Married,ON,Tillsonburg,Canada
123438,ZW31386,ZW,male,1985-12-07,active,Commonlaw,ON,London,Canada
123439,ZW31484,ZW,male,1986-03-15,active,Single,ON,Thorndale,Canada


In [ ]:
#import Owners on Database
for index, row in Owners_df.iterrows():
  cursor.execute('insert into owners (id,ident,sex,bdate,status,maritial,region,city,country) values (%s,%s,%s,%s,%s,%s,%s,%s,%s)',(row.ID,row.Ident,row.sex,row.bdate,row.status,row.marital,row.region,row.city,row.country))



In [ ]:
#commit the import
conn.commit()


In [ ]:
# Read in data from S3 Buckets - wallet
wallet_df = pd.read_csv('https://uoftfinalproject.s3.us-east-2.amazonaws.com/Online_Wallet.csv')


#Cleand Data
wallet_df=wallet_df.dropna()
wallet_df

,ID,date,amount,Mobile
0,LD30172,02/19/18 0:00,-45.14,Apple Pay
1,LD30172,03/21/18 0:00,-20.00,Samsung Pay
2,NM36985,11/28/18 0:00,-1.91,Samsung Pay
3,GJ35713,04/20/18 0:00,-1.80,Samsung Pay
4,RG34883,01/28/18 0:00,-4.66,Google Pay
...,...,...,...,...
12408,SM36977,10/02/18 0:00,-1.50,Apple Pay
12409,EP30029,03/30/18 0:00,-10.00,Google Pay
12410,EP30029,12/14/18 0:00,-1.80,Google Pay
12411,EV34503,08/26/18 0:00,-13.42,Google Pay


In [ ]:
#merge to get only owners imported
wallet_merge_df = pd.merge(wallet_df, Owners_df, on="ID")
wallet_merge_df

,ID,date,amount,Mobile,Ident,sex,bdate,status,marital,region,city,country
0,LD30172,02/19/18 0:00,-45.14,Apple Pay,LD,male,1982-08-11,active,Commonlaw,ON,Ingersoll,Canada
1,LD30172,03/21/18 0:00,-20.00,Samsung Pay,LD,male,1982-08-11,active,Commonlaw,ON,Ingersoll,Canada
2,LD30172,03/04/18 0:00,-7.98,Samsung Pay,LD,male,1982-08-11,active,Commonlaw,ON,Ingersoll,Canada
3,LD30172,02/14/18 0:00,-37.21,Apple Pay,LD,male,1982-08-11,active,Commonlaw,ON,Ingersoll,Canada
4,LD30172,03/20/18 0:00,-18.14,Samsung Pay,LD,male,1982-08-11,active,Commonlaw,ON,Ingersoll,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
8272,ER33144,09/06/18 0:00,-19.03,Google Pay,ER,male,1990-09-30,active,Married,ON,Teeswater,Canada
8273,RG35895,08/02/18 0:00,-10.14,Apple Pay,RG,male,1998-04-12,active,Single,ON,Sarnia,Canada
8274,AT34315,09/25/18 0:00,-6.76,Apple Pay,AT,female,1993-12-14,active,,ON,London,Canada
8275,AT34315,09/26/18 0:00,-4.61,Apple Pay,AT,female,1993-12-14,active,,ON,London,Canada


In [ ]:
#Import Wallet
for index, row in wallet_merge_df.iterrows():
  cursor.execute('insert into walletmobile (id,date,mobile,amount) values (%s,%s,%s,%s)',(row.ID,row.date,row.Mobile,row.amount))

#commit the import
conn.commit()

In [ ]:
# Read in data from S3 Buckets - online_Autpay
online_Autpay_df = pd.read_csv('https://uoftfinalproject.s3.us-east-2.amazonaws.com/online_debitaut.csv')


#Cleand Data
online_Autpay_df=online_Autpay_df.dropna()

#merge to get only owners imported
online_Autpay_merge_df = pd.merge(online_Autpay_df, Owners_df, on="ID")
online_Autpay_merge_df


,ID,code,date,amount,Ident,sex,bdate,status,marital,region,city,country
0,SL32722,OnlinePayment,08/12/18 0:00,-141.25,SL,male,1989-08-04,active,Commonlaw,ON,Amherstburg,Canada
1,SL32722,OnlinePayment,08/05/18 0:00,-250.99,SL,male,1989-08-04,active,Commonlaw,ON,Amherstburg,Canada
2,SL32722,OnlinePayment,07/29/18 0:00,-140.00,SL,male,1989-08-04,active,Commonlaw,ON,Amherstburg,Canada
3,SL32722,OnlinePayment,07/15/18 0:00,-141.25,SL,male,1989-08-04,active,Commonlaw,ON,Amherstburg,Canada
4,SL32722,OnlinePayment,06/24/18 0:00,-216.53,SL,male,1989-08-04,active,Commonlaw,ON,Amherstburg,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
181341,NK17371,OnlinePayment,11/12/18 0:00,-50.85,NK,female,1947-07-25,active,Married,ON,Goderich,Canada
181342,AH24130,OnlinePayment,09/16/18 0:00,-1500.00,AH,female,1966-01-25,active,Commonlaw,ON,Staples,Canada
181343,EB16564,OnlinePayment,11/12/18 0:00,-121.55,EB,female,1945-05-09,active,Married,ON,Sarnia,Canada
181344,EB16564,OnlinePayment,11/12/18 0:00,-100.00,EB,female,1945-05-09,active,Married,ON,Sarnia,Canada


In [ ]:
#Import AutoPay
for index, row in online_Autpay_merge_df.iterrows():
  cursor.execute('insert into online_autpay (id,date,code,amount) values (%s,%s,%s,%s)',(row.ID,row.date,row.code,row.amount))

#commit the import
conn.commit()

In [ ]:
# Read in data from S3 Buckets - Tap_POS
Tap_POS_df = pd.read_csv('https://uoftfinalproject.s3.us-east-2.amazonaws.com/Tap_POS.csv')


#Cleand Data
Tap_POS_df=Tap_POS_df.dropna()

#merge to get only owners imported
Tap_POS_merge_df = pd.merge(Tap_POS_df, Owners_df, on="ID")
Tap_POS_merge_df

,ID,code,date,amount,Ident,sex,bdate,status,marital,region,city,country
0,KT24916,POS,08/23/18 0:00,-16.23,KT,male,1968-03-21,active,Married,ON,Harrow,Canada
1,KT24916,Tap,04/15/18 0:00,-2.09,KT,male,1968-03-21,active,Married,ON,Harrow,Canada
2,KT24916,Tap,04/15/18 0:00,-30.00,KT,male,1968-03-21,active,Married,ON,Harrow,Canada
3,KT24916,POS,09/20/18 0:00,-14.15,KT,male,1968-03-21,active,Married,ON,Harrow,Canada
4,KT24916,POS,09/20/18 0:00,-21.54,KT,male,1968-03-21,active,Married,ON,Harrow,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
562391,ES27052,POS,08/24/18 0:00,-32.71,ES,female,1974-01-25,active,Married,ON,Thedford,Canada
562392,AF26963,POS,08/24/18 0:00,-85.00,AF,female,1973-10-28,active,Married,ON,London,Canada
562393,EH14585,POS,11/12/18 0:00,-15.29,EH,male,1939-12-08,active,Married,ON,Arkona,Canada
562394,SC27645,POS,11/12/18 0:00,-789.87,SC,male,1975-09-10,active,Married,ON,Ingersoll,Canada


In [ ]:
#Import Tap_POS
for index, row in Tap_POS_merge_df.iterrows():
  cursor.execute('insert into tappos (id,date,code,amount) values (%s,%s,%s,%s)',(row.ID,row.date,row.code,row.amount))

#commit the import
conn.commit()

In [ ]:
# Read in data from S3 Buckets - ATM_payment
ATM_payment_df = pd.read_csv('https://uoftfinalproject.s3.us-east-2.amazonaws.com/ATM_payment.csv')


#Cleand Data
ATM_payment_df=ATM_payment_df.dropna()

#merge to get only owners imported
ATM_payment_merge_df = pd.merge(ATM_payment_df, Owners_df, on="ID")
ATM_payment_merge_df


,ID,code,date,amount,Ident,sex,bdate,status,marital,region,city,country
0,DP15610,ATMPayment,08/23/18 0:00,-800.00,DP,male,1942-09-28,active,Married,ON,Clinton,Canada
1,NV12664,ATMPayment,11/12/18 0:00,-192.29,NV,female,1934-09-04,active,Widowed,ON,Ilderton,Canada
2,NV12664,ATMPayment,11/12/18 0:00,-32.88,NV,female,1934-09-04,active,Widowed,ON,Ilderton,Canada
3,NV12664,ATMPayment,11/12/18 0:00,-102.26,NV,female,1934-09-04,active,Widowed,ON,Ilderton,Canada
4,NV12664,ATMPayment,11/04/18 0:00,-130.80,NV,female,1934-09-04,active,Widowed,ON,Ilderton,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
10361,WW20622,ATMPayment,11/12/18 0:00,-200.00,WW,male,1956-06-18,active,Divorced,ON,Mount Hope,Canada
10362,WW20622,ATMPayment,11/12/18 0:00,-85.88,WW,male,1956-06-18,active,Divorced,ON,Mount Hope,Canada
10363,WW20622,ATMPayment,11/12/18 0:00,-300.00,WW,male,1956-06-18,active,Divorced,ON,Mount Hope,Canada
10364,DW34055,ATMPayment,08/25/18 0:00,-135.21,DW,male,1993-03-29,active,Single,ON,Leamington,Canada


In [ ]:
#Import ATM_payment
for index, row in ATM_payment_merge_df.iterrows():
  cursor.execute('insert into ATM_payments (id,date,code,amount) values (%s,%s,%s,%s)',(row.ID,row.date,row.code,row.amount))

#commit the import
conn.commit()

In [ ]:
#Close the Cursor
cursor.close()